# Imports

In [ ]:
# General
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import string
import re
import os
import tqdm
import gc
import spacy
# Load spacy model for lemmatization
nlp = spacy.load("en_core_web_sm")

# Preprocessing
from sklearn.preprocessing import LabelEncoder

from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec
from nltk.tokenize import regexp_tokenize

# Models

import torch
from nltk.tokenize import regexp_tokenize
# Evaluation metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
# df_train  = pd.read_parquet('/kaggle/input/data-95/train_95.parquet')
# print(df_train.head())

# Read Json data and convert it to parqet format

In [ ]:
# import pandas as pd

# # Load JSON dataset
# df = pd.read_json('/kaggle/input/nlp-pizzaa-ner-dataset/PIZZA_train.json')

# # Save as Parquet
# df.to_parquet('/kaggle/working/PIZZA_train_all.parquet', index=False)

# print("JSON converted to Parquet!")


In [ ]:
tqdm.tqdm.pandas()

# Split Test and Train Data

In [ ]:
# import pandas as pd
# from sklearn.model_selection import train_test_split

# # Load your dataset
# df = pd.read_parquet('/kaggle/input/train-parquet/train.parquet')

# # Shuffle the dataset
# df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# # Define features and target
# X = df['src']  
# Y = df['top']  

# # Split into train and test sets
# X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.05, random_state=42)

# print("Data shuffled and split into training and testing sets.")


# Save Test and Train data

In [ ]:
# # Combine X_train and y_train into a single DataFrame
# df_train = pd.DataFrame({'src': X_train, 'top': y_train})
# df_test = pd.DataFrame({'src': X_test, 'top': y_test})

# # Save to the dataset folder
# df_train.to_parquet('/kaggle/working/train_95.parquet', index=False)
# df_test.to_parquet('/kaggle/working/test_5.parquet', index=False)

# print("Train and test datasets saved as Parquet files.")


# Tokenize SRC and save it

In [ ]:

# # df_train = pd.read_parquet("train.parquet")
# token_pattern=r"(?u)\b\w+(?:'\w+)?(?:-\w+)*\b"
# df_train["tokenized"] = df_train["src"].progress_apply(lambda x: regexp_tokenize(x, token_pattern)) 
# # print(df_train["tokenized"])
# df_train.to_parquet('/kaggle/working/train_95.parquet', index=False)
# print("Tokenized src data saved to parquet file.")


# Tranform BIO tags to numbers

In [ ]:
# # from sklearn.preprocessing import LabelEncoder
# full_text = " ".join(df_train['top'].to_list())
# entities = [x.group() for x in re.finditer("(?<=\()[A-Z]+(_[A-Z]+)*", full_text)]
# entities = list(set(entities)) # Unique

#  # Using BIO Tagging
# bio_entities = [f"{letter}-{entity}" for entity in entities for letter in "BI"]
# bio_entities.append('O')
# bio_entities

# # label_encoder = LabelEncoder()
# # label_encoder.fit(bio_entities)
# print(len(bio_entities))
# print(bio_entities)

# Extract TOP Target IS and Save it 

In [ ]:
# def extract_labels(top: str, entities):
#     # Extract words and parenthesis
#     pattern = r"\b\w+(?:'\w+)?(?:-\w+)*\b|[()]"
#     tokens = regexp_tokenize(top, pattern)
    
#     labels = []
#     count = 0
  
#     is_beginning = True
#     order_type = "PIZZAORDER"
#     for i, token in enumerate(tokens):
       
#         if token in entities and token not in ["PIZZAORDER", "DRINKORDER"]:
#             continue
#         elif token == "(":
#             count += 1
#         elif token == ")":
#             count -= 1
#         elif token == "PIZZAORDER":
#             order_type = "PIZZAORDER"
#         elif token == "DRINKORDER":
#             order_type = "DRINKORDER"
        
#         elif count == 0:
#             labels.append("O")
#             is_beginning = True
#         else:
#             if is_beginning == True:
#                 labels.append("B-" + order_type)
#                 is_beginning = False
#                 continue
#             if is_beginning == False:
#                 labels.append("I-" + order_type)
#     labels = label_encoder.transform(labels)
#     return labels


# # Extract one sample from the 'train.TOP' column (for example, the first entry)



In [ ]:
# Function to apply extract_labels to each row
def apply_extract_labels(row, entities):
    # Extract labels for each row's 'top' column
    labels = extract_labels(row['top'], entities)
    
    # Return the labels (make sure it's in a proper format for storing in DataFrame)
    return labels.tolist()

# # Apply the function to each row in the 'top' column and store the result in a new column 'IS_labels'
# df_train['IS_labels'] = df_train.apply(lambda row: apply_extract_labels(row, entities), axis=1)

# # Check the result

# print(df_train[['top', 'IS_labels']].head())

# # Save the modified DataFrame to a new Parquet file
# df_train.to_parquet('train_95.parquet', index=False)

# print("Data with IS_labels saved to train_95")

In [ ]:
# print(df_train[['src','tokenized','top', 'IS_labels']].head())

# convert parquet to csv

In [ ]:
# import pandas as pd

# # Read the Parquet file


# # Save it as a CSV file
# df.to_csv('/kaggle/working/train_95.csv', index=False)


# word2vec

In [ ]:
DATASET_PATH = "/kaggle/input/pizza-dataset"
OUTPUT_ROOT_PATH = "/kaggle/working"
MODELS_PATH = OUTPUT_ROOT_PATH + "/models"
PYTORCH_MODELS_PATH = MODELS_PATH + "/checkpoints"

In [ ]:
os.makedirs(MODELS_PATH, exist_ok=True)

In [ ]:
# df_train = pd.read_parquet("train.parquet")
# print(df_train['tokenized'][0])
# print(df_train['tokenized'].apply(type).value_counts())
# print(df_train.columns)

In [ ]:
df_train = pd.read_parquet('/kaggle/input/data-95/train_95.parquet',columns=['tokenized'])
sentences=df_train['tokenized'].tolist()


# Convert each numpy array in the list to a Python list
sentences = [sentence.tolist() if isinstance(sentence, np.ndarray) else sentence for sentence in sentences]
'''  
<class 'list'>
<class 'list'>
<class 'str'>
['party', 'size', 'dried', 'peppers', 'pizza', 'and', 'a', 'sprite']
'''
# Verify the format
print(type(sentences))         # Should be list
print(type(sentences[0]))      # Should be list
print(type(sentences[0][0]))   # Should be str
print(sentences[0])            # Check the first sentence



# Train Word2Vec model
word2vec_model = Word2Vec(sentences=sentences, vector_size=200, window=5, min_count=1, workers=4)
print(word2vec_model)

In [ ]:
# max_length = max(len(sentence) for sentence in df_train['IS_labels']) 
# print(max_length) 

In [ ]:
# import os

# # Specify the file path you want to delete
# file_path = '/kaggle/working/word2vec_features.npy'  # Replace with the path of the file you want to delete

# # Check if the file exists before deleting it
# if os.path.exists(file_path):
#     os.remove(file_path)
#     print(f"File {file_path} has been deleted.")
# else:
#     print(f"File {file_path} not found.")


In [ ]:


DATASET_PATH = "/kaggle/input/pizza-dataset"
OUTPUT_ROOT_PATH = "/kaggle/working"
MODELS_PATH = OUTPUT_ROOT_PATH + "/models"
PYTORCH_MODELS_PATH = MODELS_PATH + "/checkpoints"

# General
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import string
import re
import os
import tqdm

import spacy
# Load spacy model for lemmatization
nlp = spacy.load("en_core_web_sm")

# Preprocessing
from sklearn.preprocessing import LabelEncoder

from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec
from nltk.tokenize import regexp_tokenize

# Models

import torch

# Evaluation metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score



In [ ]:
# # File paths
# parquet_file_path = '/kaggle/working/train_95.parquet'
# model_file_path = '/kaggle/input/word2vecmodel/word2vec_model.bin'

# # Load the Parquet file
# train_data = pd.read_parquet(parquet_file_path)
# print("Parquet file loaded successfully.")

# # Load the Word2Vec model
# word2vec_model = Word2Vec.load(model_file_path)
# print("Word2Vec model loaded successfully.")

# show variables taking much space in the ram

In [ ]:
# import sys
# import pandas as pd

# # Create a snapshot of the global namespace to prevent changes during iteration
# global_vars = globals().copy()

# # Create a list to store variable information
# variables = []

# # Loop through all variables in the global namespace
# for var_name, var_value in global_vars.items():
#     try:
#         size = sys.getsizeof(var_value)
#         variables.append({'Variable': var_name, 'Size (Bytes)': size})
#     except Exception as e:
#         variables.append({'Variable': var_name, 'Size (Bytes)': 'Unknown'})

# # Convert to a DataFrame for easy viewing
# variables_df = pd.DataFrame(variables).sort_values(by='Size (Bytes)', ascending=False)

# # Show top 10 largest variables
# print(variables_df)


In [ ]:
# # List of variables you want to delete
# vars_to_delete = ['df', 'train_df', 'df_train', 'Y', 'y_train', 'full_text', 'X_train', 'X', 'df_test','y_test','X_test','variables_df']

# # Delete each variable in the list
# for var_name in vars_to_delete:
#     if var_name in globals():
#         del globals()[var_name]

# # Verify that only the 'train_data' variable remains
# print(globals().keys())  # This should only show 'train_data'


# show memory consumption

In [ ]:
# import psutil

# # Get the current system memory status
# memory = psutil.virtual_memory()

# # Display total memory, used memory, and available memory
# print(f"Total Memory: {memory.total / (1024 ** 3):.2f} GB")
# print(f"Used Memory: {memory.used / (1024 ** 3):.2f} GB")
# print(f"Free Memory: {memory.available / (1024 ** 3):.2f} GB")
# print(f"Memory Percentage Used: {memory.percent}%")


In [ ]:

# Load tokenized data (assuming you have 'tokenized' column in the train.parquet)
# train_data = pd.read_parquet("train.parquet", columns=["tokenized", "IS_labels"])
# train_data['tokenized'] = train_data['tokenized'].apply(lambda x: x.tolist() if isinstance(x, np.ndarray) else x)
# Apply the get_word_vectors function to get word vectors for each sentence
# train_data['word_vectors'] = train_data['tokenized'].apply(lambda x: get_word_vectors(x, word2vec_model)) 



# LSTM

## checkpoint


In [ ]:
def save_checkpoint(model, optimizer, epoch, loss, file_path):
    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss
    }
    torch.save(checkpoint, file_path)
    print(f"Checkpoint saved at {file_path}")

## Load the data

## Define LSTM in pytorch

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1, dropout=0.2):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        # LSTM layer
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        # Fully connected layer for classification
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # Pass input through LSTM
        lstm_out, _ = self.lstm(x)
        # Use the output of the last time step for classification
        out = self.fc(lstm_out)  # Shape: (batch_size, output_size)
        return out


# Define DataSet Class

In [ ]:
import pandas as pd
from torch.utils.data import Dataset
import numpy as np
import torch
from tensorflow.keras.preprocessing.sequence import pad_sequences
from gensim.models import Word2Vec

class CustomDataSet(Dataset):
    def __init__(self, file,model):
        df_train = pd.read_parquet(file,columns=['tokenized','IS_labels'])
        self.sentences = df_train['tokenized']
        self.labels = df_train['IS_labels']
        self.feature_exctractor_model = model
       

    def __len__(self):
        return len(self.labels)
        
    def get_sentence_vectors(self, sentence):
        """
        Convert a tokenized sentence into a list of word vectors using the Word2Vec model.
        
        :param sentence: List of tokens.
        :return: List of word vectors (numpy array).
        """
        sentence_vectors = []
        for word in sentence:
            if word in self.feature_exctractor_model.wv.key_to_index:
                sentence_vectors.append(self.feature_exctractor_model.wv[word])  # Word2Vec vector
            else:
                sentence_vectors.append(np.zeros(self.vector_size))  # Zero vector for unknown words
        # print("Sentence vectors:", sentence_vectors[:5])  # Displaying a few vectors
        return sentence_vectors

    def __getitem__(self, idx):
        # Get the sentence vector
        feature = self.get_sentence_vectors(self.sentences.iloc[idx])
        # Apply padding to the sentence vectors
        padded_feature = pad_sequences([feature], maxlen=50, dtype='float32', padding='post')
        
        # For sequence labels, pad them as well
        label = self.labels.iloc[idx]
        padded_label = pad_sequences([label], maxlen=50, padding='post', value=-1)  # Padding with -1 for labels
        return padded_feature[0], padded_label[0]


# Train Loop

In [ ]:
import torch
import gc
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader
import torch.nn as nn
from tensorflow.keras.preprocessing.sequence import pad_sequences
from gensim.models import Word2Vec

# Assuming CustomDataSet and LSTMModel are defined as you provided

def save_checkpoint(model, optimizer, epoch, loss, file_path):
    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss
    }
    torch.save(checkpoint, file_path)
    print(f"Checkpoint saved at {file_path}")

# Training loop
def train_model(file,feature_extractor ,model, num_epochs=10, batch_size=32, learning_rate=0.001, chunk_size=10000):
    # Load the dataset
    dataset = CustomDataSet(file,feature_extractor )
    train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    
    # Model, loss function, and optimizer initialization
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    criterion = nn.CrossEntropyLoss(ignore_index=-1)  # Assuming -1 is the padding index
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    # Training loop
    for epoch in range(num_epochs):
        epoch_loss = 0
        total_correct_train = 0
        total_samples_train = 0
        print(f"Epoch {epoch + 1}/{num_epochs}")

        model.train()
        
        for batch_idx, (sentences_batch, labels_batch) in enumerate(tqdm(train_loader, desc=f"Training Epoch {epoch+1}", unit="batch")):
            # Move batch data to the appropriate device
            sentences_batch = sentences_batch.to(device)
            labels_batch = labels_batch.to(device)
            # print("sentences batch size :",sentences_batch.shape)
            # print("labels_batch batch size :",labels_batch.shape)
            
           
            # Forward pass
            outputs = model(sentences_batch)  # Shape: (batch_size, seq_len, num_classes)
            
            # Flatten the output and labels for CrossEntropyLoss
            outputs_flat = torch.flatten(outputs, start_dim=0, end_dim=1)  # (batch_size * seq_len, num_classes)
            targets_flat = labels_batch.view(-1).long()  # (batch_size * seq_len)
            # print("output size :",outputs_flat.shape)
            # print("targets_flat size :",targets_flat.shape)
            # Compute loss
            batch_loss = criterion(outputs_flat, targets_flat)
            optimizer.zero_grad()
            batch_loss.backward()
            optimizer.step()

            # Accumulate loss
            epoch_loss += batch_loss.item()

            # Accuracy calculation
            predictions = outputs.argmax(-1)  # (batch_size, seq_len)
            mask = labels_batch != -1  # Exclude padding from accuracy calculation
            total_correct_train += (predictions[mask] == labels_batch[mask]).sum().item()
            total_samples_train += mask.sum().item()  # Count valid labels (non-padding)

        # Calculate average loss and accuracy
        epoch_loss /= len(train_loader)  # Average loss per batch
        epoch_acc = total_correct_train / total_samples_train  # Accuracy: correct / total valid labels

        print(f'Epoch [{epoch + 1}/{num_epochs}] | Loss: {epoch_loss:.4f} | Accuracy: {epoch_acc:.4f}')
        
        # Save the model after every epoch
        save_checkpoint(model, optimizer, epoch, epoch_loss, f"model_epoch_{epoch+1}.pth")

        # Clear GPU cache and collect garbage
        gc.collect()
        torch.cuda.empty_cache()

    print("Training complete!")

# Example usage:
# Assuming you have a pre-trained Word2Vec model named `word2vec_model`
# word2vec_model 
# # TODO : CHANGE THIS 24 TO 5 :   B-PIZZA ,I-PIZZA ,B-DRINK , I-DRINK , O
# train_model("/kaggle/input/data-95/train_95.parquet",word2vec_model ,LSTMModel(input_size=200, hidden_size=128, output_size=24), num_epochs=10)


# NER

In [ ]:

# #full_entities = np.load('/kaggle/input/ner-entities/full_entities.npy')
# # Print the data or its shape
# print("full entities :")
# print(full_entities)


# # Using BIO Tagging
# full_bio_entities = [f"{letter}-{entity}" for entity in full_entities for letter in "BI"]
# full_bio_entities.append('O')
# full_bio_entities.append('B-COMPLEX_QUANTITY')
# full_bio_entities.append('I-COMPLEX_QUANTITY')
# full_bio_entities.remove('I-PIZZAORDER')
# full_bio_entities.remove('B-PIZZAORDER')
# full_bio_entities.remove('B-DRINKORDER')
# full_bio_entities.remove('I-DRINKORDER')
# label_encoder = LabelEncoder()
# label_encoder.fit(full_bio_entities)

# print(full_bio_entities)
# def extract_NER_labels(top: str, entities):
#     # Extract words and parenthesis
#     pattern = r"\b\w+(?:'\w+)?(?:-\w+)*\b|[()]"
#     tokens = regexp_tokenize(top, pattern)
    
#     labels = []
#     count = 0
#     not_str =""
#     complex_topping_begin = False 
#     is_beginning = True
#     order_type = ""
#     for token in tokens:
#         if token in ["PIZZAORDER", "DRINKORDER",'ORDER']:
#             count -= 1
#             continue
#         elif token == "(":
#             count += 1
#         elif token == ")":
#             count -= 1
#             if count == 0:
#                 is_beginning = True
#                 complex_topping_begin = False
#                 not_str = ""
#                 order_type = ""
#             if count < 0:
#                 count = 0
#         elif token == "COMPLEX_TOPPING":
#             order_type = "COMPLEX_TOPPING"
#             complex_topping_begin = True
#         elif token == "NOT":
#             not_str = "NOT_"
#         elif token in entities:
#             order_type = token
#         elif count == 0:
#             labels.append("O")
           
#         else:
#             if complex_topping_begin:
#                 if is_beginning:
#                     labels.append("B-COMPLEX_" + order_type)
#                     is_beginning = False
#                     continue
#                 else:
#                     labels.append("I-COMPLEX_" + order_type)
#             if is_beginning:
#                 labels.append("B-" + not_str + order_type)
#                 is_beginning = False
#                 continue
#             else:
#                 labels.append("I-" + not_str + order_type) 
#     labels = label_encoder.transform(labels)
#     return labels


In [ ]:
# # Function to apply extract_labels to each row
# def apply_extract_ner_labels(row, entities):
#     # Extract labels for each row's 'top' column
#     labels = extract_NER_labels(row['top'], entities)
    
#     # Return the labels (make sure it's in a proper format for storing in DataFrame)
#     return labels

# # # Apply the function to each row in the 'top' column and store the result in a new column 'IS_labels'
# df_train = pd.read_parquet('/kaggle/input/data-95/train_95.parquet')
# df_train['NER_labels'] = df_train.apply(lambda row: apply_extract_ner_labels(row, full_entities), axis=1)
# print(df_train.head())
# # Check the result



# # Save the modified DataFrame to a new Parquet file
# #df_train.to_parquet('train_95.parquet', index=False)

# #print("Data with IS_labels saved to train_95")

In [ ]:
# print(len(full_bio_entities))
# df_train.to_parquet('/kaggle/working/train_95_with_ner.parquet', index=False)
# # 

In [ ]:
# df_train = pd.read_parquet('/kaggle/working/train_95_with_ner.parquet') 
# print(df_train.columns)

In [ ]:
# row_values = df_train[['top', 'NER_labels','src','tokenized']].iloc[0]
# print(row_values['top'])         # Value from 'top'
# print(row_values['NER_labels'])  # Value from 'NER_labels'
# print(row_values['tokenized'])  # Value from 'NER_labels'
# print(df_train[['top', 'NER_labels']].head())

# NER DataSet

In [ ]:
import pandas as pd
from torch.utils.data import Dataset
import numpy as np
import torch
from tensorflow.keras.preprocessing.sequence import pad_sequences
from gensim.models import Word2Vec

class CustomDataSet(Dataset):
    def __init__(self, file,model):
        df_train = pd.read_parquet(file,columns=['tokenized','NER_labels'])
        self.sentences = df_train['tokenized']
        self.labels = df_train['NER_labels']
        self.feature_exctractor_model = model
       

    def __len__(self):
        return len(self.labels)
        
    def get_sentence_vectors(self, sentence):
        """
        Convert a tokenized sentence into a list of word vectors using the Word2Vec model.
        
        :param sentence: List of tokens.
        :return: List of word vectors (numpy array).
        """
        sentence_vectors = []
        for word in sentence:
            if word in self.feature_exctractor_model.wv.key_to_index:
                sentence_vectors.append(self.feature_exctractor_model.wv[word])  # Word2Vec vector
            else:
                sentence_vectors.append(np.zeros(self.vector_size))  # Zero vector for unknown words
        # print("Sentence vectors:", sentence_vectors[:5])  # Displaying a few vectors
        return sentence_vectors

    def __getitem__(self, idx):
        # Get the sentence vector
        feature = self.get_sentence_vectors(self.sentences.iloc[idx])
        # Apply padding to the sentence vectors
        padded_feature = pad_sequences([feature], maxlen=50, dtype='float32', padding='post')
        
        # For sequence labels, pad them as well
        label = self.labels.iloc[idx]
        padded_label = pad_sequences([label], maxlen=50, padding='post', value=-1)  # Padding with -1 for labels
        return padded_feature[0], padded_label[0]


# Training Loop NER

In [ ]:
import torch
import gc
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader
import torch.nn as nn
from tensorflow.keras.preprocessing.sequence import pad_sequences
from gensim.models import Word2Vec

# Assuming CustomDataSet and LSTMModel are defined as you provided

def save_checkpoint(model, optimizer, epoch, loss, file_path):
    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss
    }
    torch.save(checkpoint, file_path)
    print(f"Checkpoint saved at {file_path}")

# Training loop
def train_model(file,feature_extractor ,model, num_epochs=10, batch_size=32, learning_rate=0.001, chunk_size=10000):
    # Load the dataset
    dataset = CustomDataSet(file,feature_extractor )
    train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    
    # Model, loss function, and optimizer initialization
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    criterion = nn.CrossEntropyLoss(ignore_index=-1)  # Assuming -1 is the padding index
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    # Training loop
    for epoch in range(num_epochs):
        epoch_loss = 0
        total_correct_train = 0
        total_samples_train = 0
        print(f"Epoch {epoch + 1}/{num_epochs}")

        model.train()
        
        for batch_idx, (sentences_batch, labels_batch) in enumerate(tqdm(train_loader, desc=f"Training Epoch {epoch+1}", unit="batch")):
            # Move batch data to the appropriate device
            sentences_batch = sentences_batch.to(device)
            labels_batch = labels_batch.to(device)
            # print("sentences batch size :",sentences_batch.shape)
            # print("labels_batch batch size :",labels_batch.shape)
            
           
            # Forward pass
            outputs = model(sentences_batch)  # Shape: (batch_size, seq_len, num_classes)
            
            # Flatten the output and labels for CrossEntropyLoss
            outputs_flat = torch.flatten(outputs, start_dim=0, end_dim=1)  # (batch_size * seq_len, num_classes)
            targets_flat = labels_batch.view(-1).long()  # (batch_size * seq_len)
            # print("output size :",outputs_flat.shape)
            # print("targets_flat size :",targets_flat.shape)
            # Compute loss
            batch_loss = criterion(outputs_flat, targets_flat)
            optimizer.zero_grad()
            batch_loss.backward()
            optimizer.step()

            # Accumulate loss
            epoch_loss += batch_loss.item()

            # Accuracy calculation
            predictions = outputs.argmax(-1)  # (batch_size, seq_len)
            mask = labels_batch != -1  # Exclude padding from accuracy calculation
            total_correct_train += (predictions[mask] == labels_batch[mask]).sum().item()
            total_samples_train += mask.sum().item()  # Count valid labels (non-padding)

        # Calculate average loss and accuracy
        epoch_loss /= len(train_loader)  # Average loss per batch
        epoch_acc = total_correct_train / total_samples_train  # Accuracy: correct / total valid labels

        print(f'Epoch [{epoch + 1}/{num_epochs}] | Loss: {epoch_loss:.4f} | Accuracy: {epoch_acc:.4f}')
        
        # Save the model after every epoch
        save_checkpoint(model, optimizer, epoch, epoch_loss, f"model_epoch_{epoch+1}.pth")

        # Clear GPU cache and collect garbage
        gc.collect()
        torch.cuda.empty_cache()

    print("Training complete!")

# Example usage:
# Assuming you have a pre-trained Word2Vec model named `word2vec_model`
word2vec_model 
train_model("/kaggle/input/dataset-95-with-ner-labels/train_95_with_ner.parquet",word2vec_model ,LSTMModel(input_size=200, hidden_size=128, output_size=25), num_epochs=10)
